In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/procesamiento-lenguaje-natural"

'1a - word2vec.ipynb'	     'Ejercicio 3.ipynb'      preprocessed_data.csv.zip
 bot_basado_en_reglas.ipynb   preprocessed_data.csv


Usaremos un dataset de textos de Edgar Allan Poe

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
#df = pd.read_csv('stephenking.txt', sep='    ', header=None)
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/procesamiento-lenguaje-natural/preprocessed_data.csv')

df.head()

,title,text,wikipedia_title,publication_date,first_published_in,classification,notes,normalized_date
0,A DESCENT INTO THE MAELSTRÖM,"The ways of God in Nature, as in Providence, a...",A Descent into the Maelström,May 1841,Graham's Magazine,Adventure,?,May 1841
1,A PREDICAMENT,"What chance, good lady, hath bereft you thus? ...",A Predicament,November 1838,Baltimore American Museum,Parody,"Companion to ""How to Write a Blackwood Article...",November 1838
2,A TALE OF JERUSALEM,Intensos rigidarn in frontern ascendere canos ...,A Tale of Jerusalem,"June 9, 1832",Philadelphia Saturday Courier,Humor,?,June 1832
3,A TALE OF THE RAGGED MOUNTAINS,"DURING the fall of the year 1827, while residi...",A Tale of the Ragged Mountains,April 1844,Godey's Lady's Book,"Adventure,Science_fiction",?,April 1844
4,BERENICE,"Dicebant mihi sodales, si sepulchrum amicae vi...",Berenice,March 1835,Southern Literary Messenger,Horror,?,March 1835


In [ ]:
df['text']

0     The ways of God in Nature, as in Providence, a...
1     What chance, good lady, hath bereft you thus? ...
2     Intensos rigidarn in frontern ascendere canos ...
3     DURING the fall of the year 1827, while residi...
4     Dicebant mihi sodales, si sepulchrum amicae vi...
                            ...                        
65    [Astounding News by Express, via Norfolk!—The ...
66    A SEQUEL TO “THE MURDERS IN THE RUE MORGUE.”  ...
67    IN speaking of the Poetic Principle, I have no...
68    BY late accounts from Rotterdam, that city see...
69    AS it is well known that the ‘wise men’ came ‘...
Name: text, Length: 70, dtype: object

In [ ]:
df = pd.DataFrame(df['text'])

In [ ]:
df

,text
0,"The ways of God in Nature, as in Providence, a..."
1,"What chance, good lady, hath bereft you thus? ..."
2,Intensos rigidarn in frontern ascendere canos ...
3,"DURING the fall of the year 1827, while residi..."
4,"Dicebant mihi sodales, si sepulchrum amicae vi..."
...,...
65,"[Astounding News by Express, via Norfolk!—The ..."
66,A SEQUEL TO “THE MURDERS IN THE RUE MORGUE.” ...
67,"IN speaking of the Poetic Principle, I have no..."
68,"BY late accounts from Rotterdam, that city see..."


In [ ]:
print("Cantidad de documentos:", df.shape[0])


Cantidad de documentos: 70


In [ ]:
import re
import unicodedata
import string

def preprocess_clean_text(text):    
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # Se eliminar cualquier caracter distinto de los del regex
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]'
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text
     

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk_stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones. Aplicamos filtros, pasamos a minuscula y eliminamos stop words
for _, row in df[:None].iterrows():
    tokens = text_to_word_sequence(row[0],filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True)
    sentence_tokens.append([a for a in tokens if a not in nltk_stop_words])

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 70


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 5853


In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=250,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 847978.0625
Loss after epoch 1: 843190.8125
Loss after epoch 2: 825766.375
Loss after epoch 3: 812451.5
Loss after epoch 4: 811687.25
Loss after epoch 5: 763060.5
Loss after epoch 6: 759536.5
Loss after epoch 7: 757787.0
Loss after epoch 8: 759465.5
Loss after epoch 9: 758160.5
Loss after epoch 10: 732765.0
Loss after epoch 11: 692842.0
Loss after epoch 12: 692919.0
Loss after epoch 13: 693214.0
Loss after epoch 14: 690788.0
Loss after epoch 15: 688832.0
Loss after epoch 16: 691030.0
Loss after epoch 17: 689205.0
Loss after epoch 18: 687486.0
Loss after epoch 19: 688927.0
Loss after epoch 20: 686860.0
Loss after epoch 21: 686444.0
Loss after epoch 22: 667896.0
Loss after epoch 23: 607142.0
Loss after epoch 24: 604646.0
Loss after epoch 25: 603872.0
Loss after epoch 26: 604550.0
Loss after epoch 27: 603554.0
Loss after epoch 28: 602200.0
Loss after epoch 29: 601970.0
Loss after epoch 30: 599198.0
Loss after epoch 31: 598802.0
Loss after epoch 32: 600212.0
Loss after 

(33257722, 41665500)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["talking"], topn=10)

[('kickapoos', 0.5048884749412537),
 ('smith', 0.5003135204315186),
 ('zee', 0.47309809923171997),
 ('berry', 0.46835803985595703),
 ('vulgar', 0.46092313528060913),
 ('circulation', 0.4598088562488556),
 ('zit', 0.4446318745613098),
 ('lose', 0.4442058503627777),
 ('brigadier', 0.4411779046058655),
 ('frog', 0.4302944839000702)]

In [ ]:
w2v_model.wv.most_similar(positive=["walking"], topn=10)

[('cane', 0.5160650610923767),
 ('way', 0.5016581416130066),
 ('pink', 0.4896237850189209),
 ('merchant', 0.47779393196105957),
 ('exchange', 0.47465211153030396),
 ('charms', 0.4672176241874695),
 ('barronitt', 0.45858556032180786),
 ('july', 0.4553198516368866),
 ('ce', 0.4440154433250427),
 ('dis', 0.4401244819164276)]

In [ ]:
w2v_model.wv.most_similar(positive=["dark"], topn=10)

[('hemisphere', 0.4619487226009369),
 ('table', 0.4097704291343689),
 ('waned', 0.408613383769989),
 ('flowed', 0.40404170751571655),
 ('dollar', 0.39764249324798584),
 ('rocks', 0.3968186378479004),
 ('mysteries', 0.39498651027679443),
 ('issue', 0.3941417932510376),
 ('sunset', 0.3895815908908844),
 ('heavily', 0.38907766342163086)]

In [ ]:
w2v_model.wv.most_similar(positive=["mysteries"], topn=10)

[('vestibule', 0.4618210196495056),
 ('hearts', 0.4575481712818146),
 ('divine', 0.4533630609512329),
 ('discoursed', 0.4512646496295929),
 ('you’re', 0.4496985673904419),
 ('anon', 0.4460476040840149),
 ('existing', 0.43353918194770813),
 ('precincts', 0.43256255984306335),
 ('notwithstanding', 0.4293864369392395),
 ('share', 0.4274493455886841)]

In [ ]:
w2v_model.wv.most_similar(positive=["grave"], topn=10)

[('moon', 0.4588015079498291),
 ('lady', 0.4418303966522217),
 ('valve', 0.4406658709049225),
 ('closer', 0.4287431538105011),
 ('building', 0.4040292501449585),
 ('appointed', 0.40371987223625183),
 ('gallant', 0.39902758598327637),
 ('sincerity', 0.3985688388347626),
 ('slumbering', 0.3969342112541199),
 ('examination', 0.3893827199935913)]

In [ ]:
w2v_model.wv.most_similar(positive=["terror"], topn=10)

[('horror', 0.5143254399299622),
 ('uttered', 0.45540472865104675),
 ('awakening', 0.4247720241546631),
 ('assembly', 0.4208236038684845),
 ('reverie', 0.4161911606788635),
 ('bewildering', 0.4097308814525604),
 ('echoes', 0.40951940417289734),
 ('overpowered', 0.40647971630096436),
 ('conceptions', 0.40567708015441895),
 ('abeyance', 0.4030711054801941)]

Cuando vemos la palabra "opuesta" a beautiful obtenemos sleep, acá tenemos un ejemplo mas claro de la trama de los textos del autor

In [ ]:
w2v_model.wv.most_similar(negative=["beautiful"], topn=10)

[('sleep', 0.1324545294046402),
 ('headlong', 0.12408821284770966),
 ('mighty', 0.12350145727396011),
 ('magic', 0.12332407385110855),
 ('device', 0.12101428210735321),
 ('inference', 0.11577296257019043),
 ('embarrassment', 0.10799449682235718),
 ('occur', 0.10712108761072159),
 ('thrilled', 0.10562478750944138),
 ('luminiferous', 0.10272025316953659)]

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Vemos algunas relaciones que tienen sentido, como door cerca de room, head cerca de face, eyes, feet. Room y open aparecen casi en el mismo lugar.
Podemos ver que death esta relativamente cerca de heart.
Vemos que alone esta cerca de appeared, still.

Se ven ejemplos donde se efectiviza el paradigma del escritor, cuyos textos son de terror, suspenso y misterio